<a href="https://colab.research.google.com/github/uswa193/ppl-prak/blob/main/progres_ml_avd_prak.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Import modul
from pandas.core.frame import DataFrame
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import scipy.stats as stats
import numpy as np
import matplotlib.pyplot as plt

#Membaca data
data = pd.read_csv("Qris.csv")
df = pd.DataFrame(data)
print(df)

#Pengecekkan missing value -- tidak ada missing value setelah code dijalankan
print("===================================================================================================")
print("Pengecekkan missing value")
dataMiss = df.isna().sum()
print(dataMiss)

df = pd.DataFrame(data)

               Timestamp                        Nama  Umur Jenis Kelamin  \
0    02/10/2023 15:16:19                       Abram    20     Laki-laki   
1    02/10/2023 15:39:19                   Airlangga    20     Laki-laki   
2    02/10/2023 15:45:11                       Wangi    20     Perempuan   
3    02/10/2023 15:51:21   Muhammad Rifki Ubaedillah    24     Laki-laki   
4    02/10/2023 16:01:56  Ni Kadek Rosa Amalia Putri    21     Perempuan   
..                   ...                         ...   ...           ...   
110  11/10/2023 18:07:55                 Dwi Sucipto    22     Laki-laki   
111  12/10/2023 21:37:13                        Lina    26     Perempuan   
112   13/10/2023 9:30:12                        aufa    21     Perempuan   
113  13/10/2023 14:05:16                        Fika    21     Perempuan   
114  13/10/2023 17:02:33                        Tyas    22     Perempuan   

    Status pekerjaan Pendapatan per bulan  \
0       Sedang studi   Kurang dari 2 juta 

In [2]:
df.rename(columns={'Apakah Anda mengetahui apa itu QRIS?':'Atensi_qris',
                   'Sejak kapan Anda mengetahui QRIS?': 'Atensi_pertama_QRIS',
                   'Seberapa sering Anda menggunakan QRIS dalam transaksi sehari-hari?': 'Frekuensi',
                   'Bagaimana pengalaman Anda menggunakan QRIS dalam bertransaksi?':'Pengalaman_pengguna' ,
                   'Apakah Anda merasa lebih efisien menggunakan QRIS dibandingkan dengan metode pembayaran lain?':'Efisiensi',
                   'Seberapa percaya Anda terhadap keamanan QRIS dalam melakukan transaksi?': 'Kepercayaan_pengguna',
                   'Apakah Anda pernah mengalami masalah keamanan (seperti kebocoran pin) saat menggunakan QRIS?':'keamanan',
                   'Apakah penggunaan QRIS meningkatkan kecenderungan Anda untuk berbelanja secara impulsif?':'Impulsif',
                   'Apakah Anda cenderung menghabiskan lebih banyak uang setelah menggunakan QRIS?':'Kencenderungan_menghabiskan_uang',
                   'Bagaimana menurut Anda pengaruh QRIS terhadap pola pengeluaran bulanan Anda?':'Pengaruh_QRIS_thd_Pengeluaran'}, inplace=True)
df.head()

,Timestamp,Nama,Umur,Jenis Kelamin,Status pekerjaan,Pendapatan per bulan,Atensi_qris,Atensi_pertama_QRIS,Frekuensi,Pengalaman_pengguna,Efisiensi,Kepercayaan_pengguna,keamanan,Impulsif,Kencenderungan_menghabiskan_uang,Pengaruh_QRIS_thd_Pengeluaran
0,02/10/2023 15:16:19,Abram,20,Laki-laki,Sedang studi,Kurang dari 2 juta,Ya,Baru-baru ini,5,4,Tidak yakin,3,Tidak,Tidak,Tidak,2
1,02/10/2023 15:39:19,Airlangga,20,Laki-laki,Sedang studi,Kurang dari 2 juta,Ya,Baru-baru ini,5,5,Ya,5,Tidak,Ya,Ya,2
2,02/10/2023 15:45:11,Wangi,20,Perempuan,Sedang studi,Kurang dari 2 juta,Ya,Sejak 2019,4,4,Ya,3,Tidak,Ya,Ya,3
3,02/10/2023 15:51:21,Muhammad Rifki Ubaedillah,24,Laki-laki,Lainnya,Kurang dari 2 juta,Ya,Sejak 2019,5,4,Tidak yakin,2,Tidak,Ya,Ya,3
4,02/10/2023 16:01:56,Ni Kadek Rosa Amalia Putri,21,Perempuan,Sedang studi,Kurang dari 2 juta,Ya,Baru-baru ini,5,5,Ya,5,Tidak yakin,Tidak yakin,Ya,3


In [3]:
df['Jenis Kelamin'] = df['Jenis Kelamin'].replace({'Laki-laki' : 0, "Perempuan" : 1})
df['Status pekerjaan'] = df['Status pekerjaan'].replace({'Sedang studi' : 0, "Bekerja" : 1, "Lainnya" : 2})
df['Pendapatan per bulan'] = df['Pendapatan per bulan'].replace({'Kurang dari 2 juta' : 0, "2-5 juta" : 1, "5-10 juta" :2, "Lebih dari 10 juta" :3})
df['Atensi_qris'] = df['Atensi_qris'].replace({'Tidak' : 0, "Ya" : 1})
df['Atensi_pertama_QRIS'] = df['Atensi_pertama_QRIS'].replace({'Baru-baru ini' : 0, "Sejak 2019" : 1})
df['Efisiensi'] = df['Efisiensi'].replace({'Tidak' : 0, "Tidak yakin" : 1, "Ya" : 2})
df['keamanan'] = df['keamanan'].replace({'Tidak' : 0, "Tidak yakin" : 1, "Ya" : 2})
df['Impulsif'] = df['Impulsif'].replace({'Tidak' : 0, "Tidak yakin" : 1, "Ya" : 2})
df['Kencenderungan_menghabiskan_uang'] = df['Kencenderungan_menghabiskan_uang'].replace({'Tidak' : 0, "Tidak yakin" : 1, "Ya" : 2})

df.head()

,Timestamp,Nama,Umur,Jenis Kelamin,Status pekerjaan,Pendapatan per bulan,Atensi_qris,Atensi_pertama_QRIS,Frekuensi,Pengalaman_pengguna,Efisiensi,Kepercayaan_pengguna,keamanan,Impulsif,Kencenderungan_menghabiskan_uang,Pengaruh_QRIS_thd_Pengeluaran
0,02/10/2023 15:16:19,Abram,20,0,0,0,1,0,5,4,1,3,0,0,0,2
1,02/10/2023 15:39:19,Airlangga,20,0,0,0,1,0,5,5,2,5,0,2,2,2
2,02/10/2023 15:45:11,Wangi,20,1,0,0,1,1,4,4,2,3,0,2,2,3
3,02/10/2023 15:51:21,Muhammad Rifki Ubaedillah,24,0,2,0,1,1,5,4,1,2,0,2,2,3
4,02/10/2023 16:01:56,Ni Kadek Rosa Amalia Putri,21,1,0,0,1,0,5,5,2,5,1,1,2,3


In [4]:
# Menggunakan OrdinalEncoder untuk mengubah data ordinal menjadi angka
ordinal_encoder = OrdinalEncoder()
df['Jenis Kelamin_encoded'] = ordinal_encoder.fit_transform(df[['Jenis Kelamin']])
df['Status pekerjaan_encoded'] = ordinal_encoder.fit_transform(df[['Status pekerjaan']])
df['Pendapatan per bulan_encoded'] = ordinal_encoder.fit_transform(df[['Pendapatan per bulan']])
df['Atensi_qris_encoded'] = ordinal_encoder.fit_transform(df[['Atensi_qris']])
df['Atensi_pertama_QRIS_encoded'] = ordinal_encoder.fit_transform(df[['Atensi_pertama_QRIS']])
df['Frekuensi_encoded'] = ordinal_encoder.fit_transform(df[['Frekuensi']])
df['Pengalaman_pengguna_encoded'] = ordinal_encoder.fit_transform(df[['Pengalaman_pengguna']])
df['Efisiensi_encoded'] = ordinal_encoder.fit_transform(df[['Efisiensi']])
df['Kepercayaan_pengguna_encoded'] = ordinal_encoder.fit_transform(df[['Kepercayaan_pengguna']])
df['keamanan_encoded'] = ordinal_encoder.fit_transform(df[['keamanan']])
df['Impulsif_encoded'] = ordinal_encoder.fit_transform(df[['Impulsif']])
df['Kencenderungan_menghabiskan_uang_encoded'] = ordinal_encoder.fit_transform(df[['Kencenderungan_menghabiskan_uang']])
df['Pengaruh_QRIS_thd_Pengeluaran_encoded'] = ordinal_encoder.fit_transform(df[['Pengaruh_QRIS_thd_Pengeluaran']])

In [5]:
# Pemilihan fitur dengan SelectKBest dan chi-squared sebagai skor
X = df[['Umur','Jenis Kelamin','Status pekerjaan_encoded','Pendapatan per bulan_encoded','Atensi_qris_encoded',
        'Atensi_pertama_QRIS_encoded','Frekuensi_encoded','Pengalaman_pengguna_encoded','Efisiensi_encoded','Kepercayaan_pengguna_encoded',
        'keamanan_encoded','Impulsif_encoded','Kencenderungan_menghabiskan_uang_encoded','Pengaruh_QRIS_thd_Pengeluaran_encoded']]
y = df['Impulsif_encoded']
# Pilih 7 fitur terbaik
k_best = SelectKBest(score_func=chi2, k=7)
X_new = k_best.fit_transform(X, y)

# Tampilkan fitur terpilih
selected_features = X.columns[k_best.get_support()]
print("Fitur terpilih:", selected_features)

Fitur terpilih: Index(['Status pekerjaan_encoded', 'Pendapatan per bulan_encoded',
       'Frekuensi_encoded', 'keamanan_encoded', 'Impulsif_encoded',
       'Kencenderungan_menghabiskan_uang_encoded',
       'Pengaruh_QRIS_thd_Pengeluaran_encoded'],
      dtype='object')


In [6]:
#Mendefinisikan ulang data dengan fitur hasil seleksi
data = df[['Status pekerjaan_encoded', 'Pendapatan per bulan_encoded',
       'Frekuensi_encoded', 'keamanan_encoded', 'Impulsif_encoded',
       'Kencenderungan_menghabiskan_uang_encoded',
       'Pengaruh_QRIS_thd_Pengeluaran_encoded']]
df = pd.DataFrame(data)
df.head()

,Status pekerjaan_encoded,Pendapatan per bulan_encoded,Frekuensi_encoded,keamanan_encoded,Impulsif_encoded,Kencenderungan_menghabiskan_uang_encoded,Pengaruh_QRIS_thd_Pengeluaran_encoded
0,0.0,0.0,4.0,0.0,0.0,0.0,1.0
1,0.0,0.0,4.0,0.0,2.0,2.0,1.0
2,0.0,0.0,3.0,0.0,2.0,2.0,2.0
3,2.0,0.0,4.0,0.0,2.0,2.0,2.0
4,0.0,0.0,4.0,1.0,1.0,2.0,2.0


In [11]:
######################## klasifikasi ##########################################
# Split data menjadi fitur (X) dan target (y)
X = df.drop(columns=['Impulsif_encoded'])
y = df['Impulsif_encoded']

# Split data menjadi data latih dan data uji
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)

# Inisialisasi model Random Forest
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Melatih model
model.fit(X_train, y_train)

# Membuat prediksi
y_pred = model.predict(X_test)

# Evaluasi model
from sklearn.metrics import accuracy_score, classification_report
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Laporan klasifikasi
print(classification_report(y_test, y_pred))


Accuracy: 0.782608695652174
              precision    recall  f1-score   support

         0.0       0.89      0.73      0.80        11
         1.0       0.33      1.00      0.50         1
         2.0       0.82      0.82      0.82        11

    accuracy                           0.78        23
   macro avg       0.68      0.85      0.71        23
weighted avg       0.83      0.78      0.80        23



In [ ]:
def predict_impulsivity_manual():
    # Meminta input dari pengguna untuk setiap fitur
    Status_pekerjaan_encoded = float(input("Status pekerjaan Anda (0-2): "))
    Pendapatan_per_bulan_encoded = float(input("Pendapatan per bulan Anda (0-3): "))
    Frekuensi_encoded = float(input("Seberapa sering Anda menggunakan QRIS (0-5): "))
    keamanan_encoded = float(input("Bagaimana keamanan QRIS (0-2): "))
    Kencenderungan_menghabiskan_uang_encoded = float(input("Bagaimaan pengaruh QRIS terhadap pengeluaran Anda (0-3): "))
    Pengaruh_QRIS_thd_Pengeluaran_encoded = float(input("Bagaimana pola pengeluaran Anda (0-2): "))

    # Membuat DataFrame baru untuk input data
    input_data = pd.DataFrame({
        'Status pekerjaan_encoded': [Status_pekerjaan_encoded],
        'Pendapatan per bulan_encoded': [Pendapatan_per_bulan_encoded],
        'Frekuensi_encoded': [Frekuensi_encoded],
        'keamanan_encoded': [keamanan_encoded],
        'Kencenderungan_menghabiskan_uang_encoded': [Kencenderungan_menghabiskan_uang_encoded],
        'Pengaruh_QRIS_thd_Pengeluaran_encoded': [Pengaruh_QRIS_thd_Pengeluaran_encoded]
    })

    # Gunakan model untuk membuat prediksi
    prediction = model.predict(input_data)

    # Tampilkan hasil prediksi
    print("Prediksi Impulsivitas Akibat QRIS:", prediction[0])

# Memanggil fungsi untuk prediksi impulsivitas dengan input manual dari pengguna
predict_impulsivity_manual()
